<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b6e71d017d6e01cc61e3f73a51fcf77bfa2b92cd53fb7527ede3582ad8f68faf
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-27 10:14:29
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.44 C
-------------------
Today PnL: 9.41 K (0.07%)
Current PnL: -22.02 L (-14.43%)
CY Booked + Current PnL: -8.13 L (-5.33%)
-------------------
Total profit:  1.80 L
Total loss:  -23.82 L
-------------------
Total Booked + Current PnL: 18.85 L (15.01%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.66%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.43 L (60.81%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.26,-13.89,16.24,0.09,14093.0,-13996.0,86781.0,101.49,51.0,M-SC,3.56,253.0,-0.99,0.60,14.01,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-2.97,-9.93,11.05,0.02,17373.0,-17339.0,157220.0,139.96,64.0,M-SC,9.09,234.0,-1.00,1.09,36.75,OX40N,NTT,PAINTS
53,NESTLEIND,1377.00,-0.00,12.56,7.85,21.40,23454.0,33345.0,298771.0,4.42,58.0,X-LC,7.68,12.0,1.42,2.08,20.04,XY25,NTT,FMCG
35,ICICIGI,2252.93,-0.66,6.54,12.82,20.19,23809.0,11401.0,185721.0,-16.49,49.0,X-MC,7.36,68.0,0.48,1.29,22.64,X40,ATH,INSURANCE
43,ITC,452.00,0.11,-1.60,12.23,10.44,24086.0,-3193.0,196945.0,-41.15,43.0,X-LC,1.02,5.0,-0.13,1.37,3.88,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,4.86,4.42,26.81,32.42,71080.0,11229.0,265125.0,-14.09,54.0,X-SC,7.50,85.0,0.16,1.84,20.19,X40,ATH,FMCG
58,RAJOOENG,143.10,3.56,-24.78,85.60,39.61,65998.0,-25400.0,77100.0,-49.12,40.0,H-SC,20.48,135.0,-0.38,0.54,8.67,AR,ATH,MISC
67,SIS,528.00,2.50,-20.94,53.38,21.26,46859.0,-23248.0,87784.0,2075.13,61.0,H-SC,3.78,166.0,-0.50,0.61,18.60,OX40N,NTT,MISC
68,SONACOMS,806.63,1.86,-10.62,56.13,39.54,50749.0,-10745.0,90414.0,-30.10,70.0,M-SC,6.64,220.0,-0.21,0.63,27.49,AR,ATH,AUTO
7,ATULAUTO,844.00,1.70,-19.16,80.07,45.57,127598.0,-37774.0,159358.0,3445.95,50.0,M-SC,4.04,244.0,-0.30,1.11,12.98,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-10.33,-13.70,110.81,81.92,109775.0,-15732.0,99066.0,-49.48,24.0,M-SC,3.34,236.0,-0.14,0.69,17.28,XR,NTT,DURABLES
39,INDIGOPNTS,1408.0,-2.97,-9.93,11.05,0.02,17373.0,-17339.0,157220.0,139.96,64.0,M-SC,9.09,234.0,-1.00,1.09,36.75,OX40N,NTT,PAINTS
28,GREENPANEL,537.0,-1.36,-32.10,110.79,43.14,137732.0,-58760.0,124318.0,178.42,33.0,M-SC,5.84,240.0,-0.43,0.86,16.94,XY24,NTT,MISC
83,VOLTAS,1530.0,-1.33,7.07,11.79,19.69,24204.0,13548.0,205290.0,-3.43,49.0,X-MC,3.33,78.0,0.56,1.43,14.34,XY25,NTT,AC
70,SURYODAY,216.0,-1.31,-19.33,52.52,23.03,75866.0,-34619.0,144452.0,57.89,49.0,H-SC,9.47,167.0,-0.46,1.00,43.33,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.06,0.47,105.75,106.72,131044.0,583.0,123919.0,-50.25,39.0,H-SC,3.08,139.0,0.00,0.86,23.96,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.57,-0.25,35.97,35.63,75193.0,-530.0,209044.0,-14.53,46.0,H-MC,3.99,119.0,-0.01,1.45,16.38,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.32,-4.50,117.71,107.91,165969.0,-6646.0,140998.0,-22.17,37.0,M-SC,10.57,216.0,-0.04,0.98,0.82,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.97,-9.93,11.05,0.02,17373.0,-17339.0,157220.0,139.96,64.0,M-SC,9.09,234.0,-1.00,1.09,36.75,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.26,-13.89,16.24,0.09,14093.0,-13996.0,86781.0,101.49,51.0,M-SC,3.56,253.0,-0.99,0.60,14.01,OX40N,NTT,DURABLES
67,SIS,528.00,2.50,-20.94,53.38,21.26,46859.0,-23248.0,87784.0,2075.13,61.0,H-SC,3.78,166.0,-0.50,0.61,18.60,OX40N,NTT,MISC
47,KANSAINER,340.00,0.06,-21.85,45.21,13.47,95273.0,-58932.0,210735.0,-67.92,33.0,H-SC,1.50,158.0,-0.62,1.47,7.15,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.30,2.93,67.36,72.27,117483.0,4962.0,174410.0,-11.52,61.0,M-LC,3.84,99.0,0.04,1.21,9.28,XR,NTT,IT
38,INDIAMART,4810.62,-0.06,0.47,105.75,106.72,131044.0,583.0,123919.0,-50.25,39.0,H-SC,3.08,139.0,0.00,0.86,23.96,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.30,2.93,67.36,72.27,117483.0,4962.0,174410.0,-11.52,61.0,M-LC,3.84,99.0,0.04,1.21,9.28,XR,NTT,IT
38,INDIAMART,4810.62,-0.06,0.47,105.75,106.72,131044.0,583.0,123919.0,-50.25,39.0,H-SC,3.08,139.0,0.00,0.86,23.96,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.57,-0.25,35.97,35.63,75193.0,-530.0,209044.0,-14.53,46.0,H-MC,3.99,119.0,-0.01,1.45,16.38,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.32,-4.50,117.71,107.91,165969.0,-6646.0,140998.0,-22.17,37.0,M-SC,10.57,216.0,-0.04,0.98,0.82,OX40N,ATH,CABLES
37,IEX,219.00,-0.42,-4.20,55.14,48.63,105623.0,-8400.0,191554.0,-36.24,53.0,H-SC,15.61,136.0,-0.08,1.33,6.87,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.08,-35.42,109.37,35.20,90877.0,-45579.0,83091.0,8.88,25.0,X-SC,13.19,92.0,-0.50,0.58,0.08,X40,NTT,FOOTWEAR
76,TMPV,600.00,-0.65,-15.43,68.11,42.18,159220.0,-42641.0,233769.0,-24.30,28.0,X-LC,1.50,3.0,-0.27,1.63,1.26,XY24,NTT,AUTO
27,GLAXO,3466.20,-0.41,-0.48,39.82,39.15,78971.0,-964.0,198320.0,-20.01,33.0,X-MC,10.09,60.0,-0.01,1.38,28.38,X40,ATH,PHARMA
55,PGHH,17907.65,0.27,-4.32,39.79,33.76,76456.0,-8670.0,192150.0,-32.51,35.0,X-MC,3.72,57.0,-0.11,1.34,0.66,X40,ATH,FMCG
46,JSWINFRA,342.00,1.35,-7.89,25.09,15.23,46302.0,-15802.0,184545.0,-21.24,35.0,X-MC,6.89,66.0,-0.34,1.28,21.82,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.11,-1.60,12.23,10.44,24086.0,-3193.0,196945.0,-41.15,43.0,X-LC,1.02,5.0,-0.13,1.37,3.88,X40,NTT,FMCG
3,ACC,3906.0,-0.56,-21.25,108.44,64.15,203206.0,-50561.0,187390.0,-54.13,54.0,X-SC,1.48,82.0,-0.25,1.30,5.12,XY24,BTT,CEMENT
76,TMPV,600.0,-0.65,-15.43,68.11,42.18,159220.0,-42641.0,233769.0,-24.30,28.0,X-LC,1.50,3.0,-0.27,1.63,1.26,XY24,NTT,AUTO
36,ICICIPRULI,790.0,0.69,3.81,26.16,30.97,47998.0,6739.0,183477.0,-20.57,59.0,X-MC,1.83,75.0,0.14,1.28,16.85,X40,ATH,INSURANCE
66,SIEMENS,4671.5,-0.96,-11.68,42.12,25.51,69224.0,-21745.0,164350.0,0.93,62.0,H-LC,1.87,49.0,-0.31,1.14,21.11,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.08,-35.42,109.37,35.20,90877.0,-45579.0,83091.0,8.88,25.0,X-SC,13.19,92.0,-0.50,0.58,0.08,X40,NTT,FOOTWEAR
55,PGHH,17907.65,0.27,-4.32,39.79,33.76,76456.0,-8670.0,192150.0,-32.51,35.0,X-MC,3.72,57.0,-0.11,1.34,0.66,X40,ATH,FMCG
54,PAGEIND,51605.08,-0.01,-4.74,32.34,26.07,50444.0,-7760.0,155980.0,-29.81,39.0,X-MC,9.36,55.0,-0.15,1.08,1.19,X40,ATH,APPARELS
76,TMPV,600.00,-0.65,-15.43,68.11,42.18,159220.0,-42641.0,233769.0,-24.30,28.0,X-LC,1.50,3.0,-0.27,1.63,1.26,XY24,NTT,AUTO
9,BAJAJHFL,181.50,1.32,-12.98,70.60,48.45,123183.0,-26026.0,174480.0,-19.23,41.0,X-MC,7.27,64.0,-0.21,1.21,1.36,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,0.56,-27.20,96.21,42.84,45532.0,-17680.0,47326.0,-52.07,43.0,X-SC,37.54,83.0,-0.39,0.33,2.98,XY24,NTT,MISC
60,RELAXO,1176.00,-0.57,-46.89,186.79,52.31,143995.0,-68071.0,77089.0,-43.47,38.0,X-SC,4.43,91.0,-0.47,0.54,3.19,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.08,-35.42,109.37,35.20,90877.0,-45579.0,83091.0,8.88,25.0,X-SC,13.19,92.0,-0.50,0.58,0.08,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.37,-5.14,28.18,21.58,37460.0,-7209.0,132930.0,-52.17,38.0,X-MC,6.56,56.0,-0.19,0.92,17.68,X40N,ATH,IT
52,MEDANTA,1486.00,-0.42,5.04,18.12,24.08,24163.0,6398.0,133348.0,-3.40,50.0,X-SC,5.54,89.0,0.26,0.93,25.64,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,0.07,-14.00,38.69,19.28,115114.0,-48427.0,297529.0,-24.92,63.0,X-LC,7.22,1.0,-0.42,2.07,9.99,X40,ATH,IT
41,INFY,2275.00,0.78,6.88,44.90,54.87,150150.0,21522.0,334410.0,-16.73,63.0,X-LC,2.62,2.0,0.14,2.33,15.73,X40,BTT,IT
76,TMPV,600.00,-0.65,-15.43,68.11,42.18,159220.0,-42641.0,233769.0,-24.30,28.0,X-LC,1.50,3.0,-0.27,1.63,1.26,XY24,NTT,AUTO
82,VBL,671.64,0.13,-5.85,44.10,35.67,131141.0,-18470.0,297372.0,-17.02,54.0,X-LC,5.77,4.0,-0.14,2.07,7.20,X40N,ATH,FMCG
43,ITC,452.00,0.11,-1.60,12.23,10.44,24086.0,-3193.0,196945.0,-41.15,43.0,X-LC,1.02,5.0,-0.13,1.37,3.88,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.76,-13.31,109.58,81.67,89110.0,-12490.0,81320.0,7311.11,50.0,L-SC,19.27,271.0,-0.14,0.57,57.48,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.86,-10.62,56.13,39.54,50749.0,-10745.0,90414.0,-30.10,70.0,M-SC,6.64,220.0,-0.21,0.63,27.49,AR,ATH,AUTO
51,MASFIN,398.61,-0.23,-6.06,29.93,22.05,27548.0,-5940.0,92040.0,-19.15,47.0,H-SC,6.98,164.0,-0.22,0.64,33.79,XR,ATH,FINANCE
70,SURYODAY,216.00,-1.31,-19.33,52.52,23.03,75866.0,-34619.0,144452.0,57.89,49.0,H-SC,9.47,167.0,-0.46,1.00,43.33,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-2.97,-9.93,11.05,0.02,17373.0,-17339.0,157220.0,139.96,64.0,M-SC,9.09,234.0,-1.00,1.09,36.75,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-2.97,-9.93,11.05,0.02,17373.0,-17339.0,157220.0,139.96,64.0,M-SC,9.09,234.0,-1.00,1.09,36.75,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.91,1.37,19.31,20.95,49283.0,3450.0,255218.0,0.35,77.0,X-LC,14.59,31.0,0.07,1.78,37.49,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-0.30,-37.38,112.23,32.89,54258.0,-28862.0,48345.0,-697.12,62.0,L-MC,5.62,259.0,-0.53,0.34,33.14,XR,NTT,BANKS
0,5PAISA,593.00,-0.09,-34.59,72.61,12.90,103772.0,-75579.0,142917.0,110.34,59.0,H-SC,13.38,161.0,-0.73,0.99,18.83,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,0.10,7.32,12.53,20.77,31485.0,17131.0,251275.0,-2.02,66.0,X-MC,3.79,62.0,0.54,1.75,14.74,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.85
1,25,44.74
2,50,75.57


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.58
MC    29.22
LC    25.23
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.84
X40      21.26
X40N     11.96
XR        9.54
AR        9.09
XY25      9.08
OX40N     7.65
X200      1.86
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.90
X-MC    22.74
X-LC    20.05
M-SC    11.96
X-SC     8.31
H-MC     4.76
H-LC     3.00
L-SC     1.41
M-MC     1.38
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.82,-4.06,37.71
IT,13.80,-13.99,72.83
FINANCE,9.44,-14.24,62.51
MISC,7.42,-21.74,71.61
ELECTRICAL,5.97,-9.27,49.29
PAINTS,5.96,-7.91,23.95
INSURANCE,4.43,0.63,34.39
PHARMA,3.87,-0.40,32.53
AUTO,3.37,-18.85,69.81


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3035835.0,21
XR,1292764.0,13
AR,1263773.0,10
X40,984434.0,14
X40N,729124.0,9
OX40N,690921.0,10
XY25,336116.0,6
SR,279566.0,2
MH,73712.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3542831.0,25
M-SC,1339345.0,15
X-MC,1212588.0,16
X-LC,883989.0,11
X-SC,725622.0,8
H-MC,393409.0,3
L-SC,260308.0,3
H-LC,125999.0,2
M-LC,117483.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1215079.0      6
           AR         868852.0      5
           XR         789649.0      7
M-SC       XY24       773093.0      6
X-MC       X40        437618.0      7
X-LC       X40        384859.0      5
X-MC       XY24       358747.0      3
H-SC       OX40N      315973.0      4
X-SC       X40N       290764.0      3
M-SC       OX40N      287556.0      5
H-SC       SR         279566.0      2
X-SC       XY24       272901.0      3
X-MC       X40N       238430.0      4
X-LC       XY24       228065.0      2
H-MC       AR         205459.0      2
X-LC       X40N       199930.0      2
H-MC       XY24       187950.0      1
X-MC       XY25       177793.0      2
L-SC       XR         172916.0      2
X-SC       X40        161957.0      2
M-SC       XR         158458.0      2
           AR         120238.0      2
M-LC       XR         117483.0      1
L-SC       OX40N       87392.0      1
H-SC       MH          73712.0      1
X-LC       XY25        71135.0      2
H-LC       AR          69224.0      1
           X200        56775.0      1
L-MC       XR          54258.0      1
M-MC       XY25        47110.0      1
L-LC       XY25        40078.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
